In [5]:
import pyclifford as pc
import numpy as np

In [4]:
pc.__version__ 

AttributeError: module 'pyclifford' has no attribute '__version__'

## Clifford Gates [Class]

`CliffordGate` class represents the Clifford gates. 
- **Attribute**:
    - qubits: a tuple containing the number of qubits that it acts on 
    - n: number of qubits it acts on
    - generator(Pauli): the generator for $C_{\pi/4}$ rotation. `Default: None`
    - forward_map(CliffordMap): forward Clifford Map. `Default: None`
    - backward_map(CliffordMap): backward Clifford Map. `Default: None`
- **Methods**:
    - `CliffordGate.reset()`: reset the Clifford gate to null
    - `CliffordGate.set_generator(Pauli)`
    - `CliffordGate.set_forward_map(CliffordMap)`
    - `CliffordGate.set_backward_map(CliffordMap)` **TODO: we need to add a handle to prevent user set conflict Clifford maps**
    - `CliffordGate.copy()`: return a copy of the Clifford gate
    - `CliffordGate.independent_from(Other_gate)`: if there is no overlap between acting qubits, then they are independent.
    - `CliffordGate.compile()`: build (forward&backward) Clifford map representation. If generator is given, then it will be converted to forward/backward map; if forward map is given, the backward map will be calculated; if backward map is given, the forward map will be calculated
    - `CliffordGate.forward(obj)`: forward transformation of the object. `obj` can be Pauli, PauliList, PauliPolynomial, StabilizerState.

CliffordGate is the low-level API(class) for Clifford gates. One can create a **null** Clifford gate by sepecifying what are the qubits this gate will act on:

Example: `circuit.CliffordGate(1,2)` will create a null gate acting on qubit 1, and 2. Null gate does not contain any `forward_map` or `backward_map`

In [44]:
gate = pc.CliffordGate(1,2)

In [45]:
print("gate acts on: {}, and there are {} qubits the gates will act on.".format(
gate.qubits, gate.n
))

gate acts on: (1, 2), and there are 2 qubits the gates will act on.


There are three ways to equip the null gate with Clifford map:
1. One can use `gate.set_generator(Pauli)`. This will create a $\pi/2$ rotation generated by the pauli string.
2. One can use `gate.set_forward_map(CliffordMap)`
3. One can use `gate.set_backward_map(CliffordMap)`

In [46]:
gate.set_generator(pc.pauli("XX"))

In [47]:
gate.set_forward_map(pc.random_clifford_map(2))

In [48]:
gate.set_backward_map(pc.random_clifford_map(2))

`gate.compile()` will construct both `forward_clifford_map` and `backward_clifford_map`. Compilation is not necessary!

`gate.copy()` will return a copy of the `CliffordGate`

One can apply Clifford Gate $U$ or $U^{\dagger}$ to: Pauli strings, Stabilizer States by `gate.forward(obj)` and `gate.backward(obj)`

In [49]:
psi = pc.ghz_state(5)
gate = pc.CliffordGate(1,2)
gate.set_generator(pc.pauli("XX"))
gate.forward(psi)

StabilizerState(
   -ZYXII
   +IZZII
   -IXYZI
   +IIIZZ
   +XXXXX)

We see the wavefunction $|\psi\rangle$ has been changed **in-place**. And the gate only changes qubit 1&2.

If two Clifford gate acting on different qubits, then they are independent. We can check dependency by `gate.independent_from(other_gate)`

In [50]:
gate1 = pc.CliffordGate(0,1)
gate2 = pc.CliffordGate(2,3)
print(gate1.independent_from(gate2))

True


There are many predefined Clifford gates. For example, H, S, X, Y, Z, CNOT. They are simply defined using the forward_map (`CliffordMap`). Here is one example:
```
def H(*qubits):
    if len(qubits)!=1:
        raise ValueError("Hadmand gate only acts on a single qubit.")
    gate = CliffordGate(*qubits)
    f_map = CliffordMap(gs = np.array([[0,1],[1,0]]),ps = np.array([0,0]))
    gate.set_forward_map(f_map)
    return gate
```

Here is a Hadmand gate example which acting on the 4th qubit:

In [51]:
pc.H(4)

[4]

There are also predefined all 24 single qubit Clifford gates. One can call `C(num, *qubits)` to generate the `num-th` Clifford gate, where `num` is the index of the gate and it is from 0-23.

In [52]:
pc.C(15,0)

[0]

Here is a CNOT gate:

In [53]:
pc.CNOT(0,1)

[0,1]

## Measurement [Class]

'''Represents a computational basis measurement.
    
**Parameters**:
* qubits: int - the qubits to be measured

**Data**:
* out: int (L) - array of measurement outcomes on corresponding qubits
     None before measurement, populated after forward measurement.
* unitary: bool - indicates that measurement is not unitary.'''

In [16]:
measure = pc.Measurement(0)
state = pc.ghz_state(5)
measure.forward(state)

(StabilizerState(
    +ZZIII
    +IZZII
    +IIZZI
    +IIIZZ
    +ZIIII),
 -1.0)

In [17]:
print("state after the measurement: ", state)

state after the measurement:  StabilizerState(
   +ZZIII
   +IZZII
   +IIZZI
   +IIIZZ
   +ZIIII)


In [19]:
print("the measurement outcome is: ", measure.out)

the measurement outcome is:  [0]


## Layers

Representes a layer of **Clifford gate** or **measurement operations**.

**Parameters**:

* ops: Operation - the operations (gates or measurements) in this layer

**Attributes**:

* prev_layer: Layer - the previous layer
* next_layer: Layer - the next layer
* forward_map: CliffordMap - the forward map of the layer (if unitary)
* backward_map: CliffordMap - the backward map of the layer (if unitary)
* unitary: bool - whether the layer is unitary

`Layer` can take `CliffordGates` and/or `Measurement` by `layer.append(objs)`:

In [20]:
circlayer = pc.Layer()

In [21]:
circlayer.append(pc.CliffordGate(0,1))
circlayer.append(pc.CliffordGate(1,3))

|[0,1][1,3]|

In [22]:
print(circlayer)

|[0,1][1,3]|


In [23]:
psi=pc.ghz_state(4)
print(psi)

StabilizerState(
   +ZZII
   +IZZI
   +IIZZ
   +XXXX)


In [24]:
circlayer.forward(psi)

(StabilizerState(
    -IYII
    -ZYZI
    +IYZY
    -YYXZ),
 0.0)

And one can also easily add all the gates/measurements in the construction step:

In [31]:
circlayer = pc.Layer(pc.CliffordGate(0,1), pc.CliffordGate(1,3))
state = pc.ghz_state(4)
print("state before the layer: ", state)
state,log2prob = circlayer.forward(state)

state before the layer:  StabilizerState(
   +ZZII
   +IZZI
   +IIZZ
   +XXXX)


In [28]:
print("state after the layer: ", state)

state after the layer:  StabilizerState(
   +IXIY
   +YIZI
   -IXZI
   -XZXX)


<div class="alert alert-block alert-success">
If the forward map and backward map for a gate is Null, then it will be assigned a <b> different </b>random clifford map each time when use calls forward() or backward()
</div>

## Clifford Circuit

`Circuit` is the high level API to assemble gates and layers, and it **automatically** calculate number of qubits in the system
It has attributes:
- first_layer
- last_layer
- forward_map
- backward_map

When `Circuit` is initialized, a null `CliffordLayer()` will be initiated.
`Circuit` can take `CliffordGate` by `circ.take(gate)`.

If the gate is independent from the current layer, it will be added. Otherwise, the circuit will create a new layer and add the gate.

Let's create a circuit with 4 qubits and generate GHZ state:

In [32]:
N = 4
circ = pc.Circuit(N)
circ.take(pc.H(0))
circ.take(pc.CNOT(0,1))
circ.take(pc.CNOT(1,2))
circ.take(pc.CNOT(2,3))
print(circ)

CliffordCircuit(
  |[2,3]|
  |[1,2]|
  |[0,1]|
  |[0]|)
 Unitary:True


Now the circuit is ready, and let's acting it to zero state to generate GHZ state.

In [33]:
state = pc.zero_state(N)
state = circ.forward(state)
print("state after circuit: ", state)

state after circuit:  StabilizerState(
   +XXXX
   +ZZII
   +IZZI
   +IIZZ)


We have successfully generated a GHZ state!

Now let's do something fun. We create a mid-circuit measurement in the GHZ circuit:

In [40]:
N = 4
circ = pc.Circuit(N)
circ.take(pc.H(0))
circ.take(pc.CNOT(0,1))
circ.measure(1)
circ.take(pc.H(1))
circ.take(pc.CNOT(1,2))
circ.take(pc.CNOT(2,3))
print(circ)

CliffordCircuit(
  |[2,3]|
  |[1,2]|
  |[1]|
  |Mz[1]|
  |[0,1]|
  |[0]|)
 Unitary:False


In [41]:
state = pc.zero_state(N)
state = circ.forward(state)
print("state after circuit: ", state)

state after circuit:  StabilizerState(
   -IXXX
   +ZXXX
   +IZZI
   +IIZZ)


In [42]:
circ.measure_result

[-1]